In [1]:
%matplotlib qt
%load_ext autoreload
%autoreload 2


In [2]:
from vpm_py.visualization import Visualizer, SliceFilter_3D, ValueFilter
from vpm_py.visualization import SliceStrategy
import os
from vpm_py.visualization import ResultPlot
import numpy as np


In [3]:
z_slice = SliceFilter_3D(strategy=SliceStrategy.MAX_INTEGRAL, plane="Z")
y_slice = SliceFilter_3D(strategy=SliceStrategy.MAX_INTEGRAL, plane="Y")
value_filter = ValueFilter(tolerance=1.5e-3, value=0.0, type="greater")

plot_particles = ResultPlot(
    "particle",
    quantity="charge",
    component="magnitude",
    filters=[],
)
plot_mesh = ResultPlot(
    "mesh",
    quantity="charge",
    component="magnitude",
    filters=[],
    options={
        "s": "auto",
    },
)

plot_mesh_quantity_z = ResultPlot(
    "mesh",
    quantity="charge",
    component="magnitude",
    filters=[z_slice],
    options={
        "add_slice_plane": plot_particles,
    },
)
plot_mesh_quantity_y = ResultPlot(
    "mesh",
    quantity="charge",
    component="magnitude",
    filters=[y_slice],
    options={
        "add_slice_plane": plot_particles,
    },
)

plotter = Visualizer(
    figure_size=(12, 10),
    plot_options=[
        plot_particles,
        plot_mesh,
        plot_mesh_quantity_z,
        plot_mesh_quantity_y,
    ],
)


------------------------------------------------------------------------------------------------------------------------------------------------------
                                                                Initializing the plot                                                                 
------------------------------------------------------------------------------------------------------------------------------------------------------
Number of plots: 4
Rows: 2, Columns: 2


In [4]:
# folder = 'results_test/'
# folder = 'results'
# folder = "/mnt/c/Users/tryfo/Videos/results_RK4"
folder = "../build/release/gnu/bin/results_deform_dt=0.01/results"
# folder = "../build/debug/gnu/bin/results_deform_CFL_variable/results"
# folder = "/mnt/c/Users/tryfo/Videos/CFL_deform/results_deform_dt=0.01/results"



In [5]:
# plotter.add_folder(
#     folder=folder,
#     particle_filename_pattern= r".*particle.*\.h5",
#     mesh_filename_pattern= r".*pm.*\.h5",
# )


In [6]:
files = os.listdir(folder)
particle_pattern = r".*particles.*\.h5"
mesh_pattern = r".*mesh.*\.h5"


# Find the files using the regex
import re

particle_regex = re.compile(particle_pattern)
mesh_regex = re.compile(mesh_pattern)

files_part = [f for f in files if particle_regex.match(f)]
files_mesh = [f for f in files if mesh_regex.match(f)]

# total_frames = len(files_part)
# for idx in np.linspace(0, len(files_part) - 1, 20, dtype=int):
#     print(f"Loading frame {idx}")
#     file_mesh = os.path.join(folder, files_mesh[idx])
#     file_part = os.path.join(folder, files_part[idx])
#     plotter.load_results_from_disk(idx, total_frames, file_part, file_mesh)

plotter.animate_result_folder(
    folder,
    total_frames=len(files_part),
    start_frame=0,
    dt=0.1,
    save_filename="animation_only_charged_particles",
    format="mp4",
    particle_filename_pattern=particle_pattern,
    mesh_filename_pattern=mesh_pattern,
)


Drawing frame 50/50: : 51it [17:21, 20.43s/it, Frame 50/50]                      


In [7]:
%matplotlib qt
import h5py
import numpy as np
filename = "/mnt/c/Users/tryfo/Videos/CFL_deform/results_deform_dt=0.01/results/00001secondary_problem.h5"
with h5py.File(filename, 'r') as f:
    # Read the dataset groups
    XS = f['X'][:]
    YS = f['Y'][:]
    ZS = f['Z'][:]
    RHS = f['RHS'][:]
    SOL = f['SOL'][:]

    XS = np.moveaxis(XS, [0, 1, 2], [2, 1, 0])
    YS = np.moveaxis(YS, [0, 1, 2], [2, 1, 0])
    ZS = np.moveaxis(ZS, [0, 1, 2], [2, 1, 0])
    RHS = np.moveaxis(RHS, [0, 1, 2, 3], [3, 2, 1, 0])
    SOL = np.moveaxis(SOL, [0, 1, 2, 3], [3, 2, 1, 0])

    # Mesh grid needs to have shape adjusted (as done in the original function)
    neq = RHS.shape[0]
    mesh_positions = np.array([XS, YS, ZS])
    mesh_solution = np.array(SOL)
    mesh_charges = np.array(RHS)

# Plot the mesh
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# Scale the point according to the charge value
# The scaling factor is arbitrary and can be adjusted
scaling = np.abs(RHS[0, :, :, :]) / np.max(np.abs(RHS[0, :, :, :]))
# Aply log scaling
scaling = 5*np.log(scaling + 1)

# Plot the mesh value of RHS (charge) and the solution
mesh = ax.scatter(XS, YS, ZS, c=RHS[0, :, :, :], cmap=cm.coolwarm, s=scaling)
plt.colorbar(mesh)
plt.show()

# Plot the mesh solution
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('X')
ax.set_ylabel('Y')

scaling = np.abs(SOL[0, :, :, :]) / np.max(np.abs(SOL[0, :, :, :]))
scaling = 2.5*np.log(scaling + 1)

# Plot the mesh value of RHS (charge) and the solution
mesh = ax.scatter(XS, YS, ZS, c=SOL[1, :, :, :], cmap=cm.coolwarm, s=scaling)
plt.colorbar(mesh)
plt.show()




In [8]:
max(RHS[1, :, :, :].flatten())


np.float32(-0.0)